In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
incomplete_n_samples = 2048
complete_n_samples = 4096

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

GPU = 3
exp_counter = '4'
loss = 'emd'
do_training = True
load_model_conf = False
do_evaluation = True

experiment_name = exp_counter + '_' + class_type + '_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
complete_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/not_scaled', \
                             str(complete_n_samples), class_syn_id)
    
n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3] 

train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

if loss == 'emd':
    max_training_epochs = 100
elif loss == 'chamfer':
    max_training_epochs = 200
    
max_evaluation_epochs = max_training_epochs

Give me the class type.
table


In [8]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [5]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=30)
assert(len(np.unique(syn_ids)) == 1)
print '%d files containing complete point clouds were found.' % (len(cmpl_pclouds), )

8509 files containing complete point clouds were found.


In [6]:
v_scan_top_dir = osp.join(top_data_dir, 'From_Matthias/shapenet_dim32_sdf_pc', class_syn_id)

if do_training or do_evaluation:
    train_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/train_models_with_scan_id.txt')
    
    train_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, train_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=30,
                                                      n_samples=incomplete_n_samples)
    
    train_data, val_data = vscan.make_validation_from_train_data(train_data, 0.10)
    
if do_evaluation:
    test_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/test_models_with_scan_id.txt')
    test_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, test_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=30,
                                                      n_samples=incomplete_n_samples)

30000 incomplete point clouds were loaded.
6000 incomplete point clouds were loaded.


In [4]:
if load_model_conf:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)],
                    'non_linearity': tf.nn.relu
                   }

    encoder_args = {'dropout_prob': 0.5}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 2,
                learning_rate = 0.0005,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.consistent_io = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

In [7]:
training_stats = []
if do_training:
    training_stats.append(ae.train(train_data, conf))

('Epoch:', '0001', 'training time (minutes)=', '5.0173', 'loss=', '351.402241205')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0002', 'training time (minutes)=', '5.1280', 'loss=', '291.668998294')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-2 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-2 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0003', 'training time (minutes)=', '5.1131', 'loss=', '277.067133699')
('Epoch:', '0004', 'training time (minutes)=', '5.1263', 'loss=', '268.133811499')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-4 is not in all_model_checkpoint_paths. Manually adding it.


...

INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-96 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0097', 'training time (minutes)=', '5.0269', 'loss=', '183.659850989')
('Epoch:', '0098', 'training time (minutes)=', '5.0147', 'loss=', '183.530279739')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-98 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-98 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0099', 'training time (minutes)=', '4.9985', 'loss=', '183.053416810')
('Epoch:', '0100', 'training time (minutes)=', '4.9918', 'loss=', '182.923624872')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-100 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/4_table_2048_4096pts_emd/models.ckpt-100 is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
max_evaluation_epochs = max_training_epochs
if do_evaluation:
    # Pick the epoch that minimizes the loss on the validation dataset.
#     n_epochs = 25 # Consider only 'n_epochs' equally spaced epochs from those that are saved.
    saved_epochs = np.array(read_saved_epochs(conf.train_dir))
    allowable_epochs = saved_epochs[saved_epochs <= max_evaluation_epochs]
#     allowable_epochs = select_first_last_and_k(allowable_epochs, n_epochs-2)
    val_stats = eval_model(ae, conf, val_data, epochs=allowable_epochs, verbose=True)
    val_loss = np.min(val_stats[:,1])
    best_epoch = int(val_stats[np.argmin(val_stats[:,1]), 0])
    print 'Best epoch = %d.' % (best_epoch,) 
        
    ae.restore_model(conf.train_dir, best_epoch)
    top_save_dir = osp.join(conf.train_dir, 'output', 'epoch_' + str(best_epoch))
    save_dir = osp.join(top_save_dir, 'test_predictions')
    test_recon, test_loss, test_feed, test_ids, test_gt = ae.evaluate(test_data, conf)
    save_reconstructions(save_dir, test_recon, test_gt, test_feed, test_ids) # save ply files of test data.    
    train_loss = ae.evaluate(train_data, conf)[1]
    
    # Report Accuracy and Coverage of test data.
    n_examples = len(test_recon)
    pred_scores = np.zeros((n_examples, 2))
    for i in xrange(n_examples):
        gt = test_gt[i]
        pred = test_recon[i]    
        pred_scores[i, 0] = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=False)
        pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02, ret_dists=False)
    
    print 'Test Median-Accuracy-Coverage:', np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])
    
    with open(osp.join(top_save_dir, 'stats.txt'), 'w') as fout:
        fout.write('Best Validation Epoch = %d\n' % (best_epoch))
        fout.write('Validation loss = %f\n' % (val_loss))
        fout.write('Train loss = %f\n' % (train_loss))
        fout.write('Test loss = %f\n' % (test_loss))
        fout.write('Gen. Error (abs, per) = %f %f\n' % (abs(test_loss-train_loss),  abs(test_loss-train_loss) / train_loss ))
        fout.write('Test Median-Accuracy-Coverage = %f %f\n' % (np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])))

Model restored in epoch 1.
[ 308.57281443]
Model restored in epoch 2.
[ 285.90274963]
Model restored in epoch 4.
[ 264.36681773]
Model restored in epoch 6.
[ 262.44297867]
Model restored in epoch 8.
[ 254.49520594]
Model restored in epoch 10.
[ 249.02615585]
Model restored in epoch 12.
[ 252.53333384]
Model restored in epoch 14.
[ 245.28909607]
Model restored in epoch 16.
[ 243.41714986]
Model restored in epoch 18.
[ 246.16073507]
Model restored in epoch 20.
[ 239.60035451]
Model restored in epoch 22.
[ 248.20402222]
Model restored in epoch 24.
[ 239.10360514]
Model restored in epoch 26.
[ 238.98098984]
Model restored in epoch 28.
[ 237.48856939]
Model restored in epoch 30.
[ 237.14779154]
Model restored in epoch 32.
[ 235.11668065]
Model restored in epoch 34.
[ 237.68522237]
Model restored in epoch 36.
[ 237.32032013]
Model restored in epoch 38.
[ 238.99141947]
Model restored in epoch 40.
[ 236.29412079]
Model restored in epoch 42.
[ 236.98200938]
Model restored in epoch 44.
[ 236.883